In [3]:
uv_path

'/opt/homebrew/bin/uv'

CalledProcessError: Command '['pip', 'install', 'docling']' returned non-zero exit status 1.

In [ ]:
import shutil
import subprocess
import sys
from typing import Sequence


def run_package_manager_command(
    args: Sequence[str],
) -> subprocess.CompletedProcess[bytes]:
    """Run a package manager command, using uv if available, otherwise falling back to pip."""
    # Check if uv is available in PATH
    uv_path = shutil.which("uv")

    if uv_path:
        # Use uv if available
        try:
            return subprocess.run(
                [uv_path] + list(args),
                check=True,
                capture_output=True,
            )
        except subprocess.CalledProcessError:
            # If uv fails, fall back to pip
            print("uv command failed, falling back to pip...")

    # Fall back to pip
    return subprocess.run(
        [sys.executable, "-m", "pip"] + list(args),
        check=True,
        capture_output=True,
    )

In [1]:
from pathlib import Path

fp = Path.cwd() / "data" / "pdf" / "DeepSeek_R1.pdf"
doc_style = Path.cwd().parent / "prompts" / "doc_style.md"

In [2]:
from lionagi import Branch, BaseModel, Field
from lionagi.tools.types import ReaderTool

flash2 = {
    "provider": "gemini",
    "model": "gemini-2.0-flash-exp",
    "limit_requests": 10,
}

flash2_think = {
    "provider": "gemini",
    "model": "gemini-2.0-flash-thinking-exp",
    "limit_requests": 10,
    "tools": "code_execution",
}

In [3]:
instruction = (
    "explain to me what the paper is about, compare with other recent papers on same "
    "discipline and provide a comparison of the results then. Taking inspirations from"
    " these, write me a complete implementation for a "
    "LLM-based agentic AI reinforcement learning framework. Must be fully functional"
    " with python 3.10+ backend, sleek type script frontend, and a docker container, "
    "with full tests, documented and ready to be deployed. try very hard and deliver"
    " the best possible implementation."
)
context = {
    "paper_path": str(fp),
    "doc_style_guide": str(doc_style),
}
instruct = {
    "instruction": instruction,
    "context": context,
}

In [4]:
a = Branch(
    chat_model=flash2_think,
    parse_model=flash2,
    tools=ReaderTool,
    use_lion_system_message=True,
)

uv command failed, falling back to pip...


ValueError: Failed to install docling: Failed to install docling: Command '['/Users/lion/lionagi/.venv/bin/python', '-m', 'pip', 'install', 'docling']' returned non-zero exit status 1.

In [ ]:
class Source(BaseModel):
    title: str
    url: str


class CodeModule(BaseModel):
    file_name: str = Field(
        description="The name of the file, possibly also and its relevant path if in a project."
    )
    code: str = Field(description="The code module content.")
    language: str = Field(
        description="The programming language the code is written in."
    )


class Documentation(BaseModel):
    file_name: str = Field(
        description="The name of the file, possibly also and its relevant path if in a project. documentations are usually in markdown format."
    )
    kind: str = Field(
        description="The kind of documentation, e.g. tutorial, API documentation, end-to-end, etc."
    )
    title: str = Field(
        default_factory=str, description="The title of the documentation."
    )
    content: str = Field(
        default_factory=str, description="The content of the documentation."
    )
    source: list[Source] | None = Field(
        default=None,
        description="The external sources of the documentation, such as website or paper, if any.",
    )


class FinalDeliverables(BaseModel):

    reflections: list[str] | str | None = Field(
        default=None, description="The reflections of the self evaluation"
    )
    milestones: list[str] | str | None = Field(
        default=None, description="The milestones of the self evaluation."
    )
    codes: list[CodeModule] | None = Field(
        default=None,
        description="The remaining code modules not yet provided.",
    )
    docs: list[Documentation] | None = Field(
        default=None,
        description="The remaining documentation not yet provided.",
    )
    source: list[Source] | None = Field(
        default=None,
        description="The external sources of the self evaluation, such as website or paper, if any.",
    )


intermediate_deliverables = [
    Documentation,
    CodeModule,
]

In [ ]:
results = []

async for i in a.ReActStream(
    instruct=instruct,
    interpret=True,
    interpret_style="exhaustively detailed",
    interpret_domain="AI",
    reasoning_effort="high",
    max_extensions=20,
    verbose=True,
    response_format=FinalDeliverables,
    intermediate_response_options=intermediate_deliverables,
):
    results.append(i)